In [29]:
from dataclasses import dataclass


@dataclass
class Config:
    pretrain_raw_data = "dataset/tamil_pretrain.txt"
    context_length = 512
    filtered_data = "dataset/filtered_cl_tamil_pretrain.txt"
    vocab_size = 30000
    tokenizer_path = "artifacts/tokenizer.json"

## Data Preprocessing

In [14]:
import mmap
import re

def clean_whitespaces(text):
    return re.sub(r"\s+", " ", text).strip()


def filter_sentences(file_path, max_len, output_file):
    with open(file_path, "r", encoding="utf-8") as f:

        mmapped_file = mmap.mmap(
            f.fileno(),
            0,
            access=mmap.ACCESS_READ
        )
        with open(output_file, "w") as out_file:
            for line in iter(mmapped_file.readline, b""):
                sentence = line.decode("utf-8").strip()
                sentence = clean_whitespaces(sentence)
                if sentence:
                    if len(sentence) >= max_len:
                        out_file.write(sentence + "\n")
        mmapped_file.close()

In [15]:
filter_sentences(Config.pretrain_raw_data, Config.context_length, "dataset/filtered_cl_tamil_pretrain.txt")

## Training a custom tokenizer
This block helps to train a BPE tokenizer from scratch for the new filtered dataset.

You can learn how to build a tokenizer from scratch block by block here: [Link](https://huggingface.co/learn/nlp-course/en/chapter6/8)

In [19]:
from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
)

In [20]:
tamil_tokenizer = Tokenizer(models.BPE())

In [21]:
tamil_tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space=False)

In [22]:
#tamil sample sentence
sample_text = "வணக்கம் உலகம்"
tamil_tokenizer.pre_tokenizer.pre_tokenize_str(sample_text)

[('à®µà®£à®ķ', (0, 3)),
 ('à¯į', (3, 4)),
 ('à®ķà®®', (4, 6)),
 ('à¯į', (6, 7)),
 ('Ġà®īà®²à®ķà®®', (7, 12)),
 ('à¯į', (12, 13))]

In [24]:
trainer = trainers.BpeTrainer(vocab_size=Config.vocab_size, special_tokens=["<|endoftext|>"])
tamil_tokenizer.model = models.BPE()
tamil_tokenizer.train([Config.filtered_data], trainer=trainer)

In [25]:
tamil_tokenizer.post_processor = processors.ByteLevel(trim_offsets=False)

In [26]:
tamil_tokenizer.decoder = decoders.ByteLevel()

In [27]:
tamil_tokenizer.decode(
    tamil_tokenizer.encode("வணக்கம் உலகம்").ids
)

'வணக்கம் உலகம்'

In [30]:
tamil_tokenizer.save(Config.tokenizer_path)

## Training pipeline setup

In [32]:
from transformers import PreTrainedTokenizerFast
from datasets import load_dataset

In [33]:
#loading the splits
data_files ={
    "train": "dataset/v2/tamil_train.txt",
    "test": "dataset/v2/tamil_test.txt",
}

raw_dataset = load_dataset(
    "text",
    data_files=data_files
)

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [35]:
tokenizer = PreTrainedTokenizerFast(
    tokenizer_file=Config.tokenizer_path,
)

outputs = tokenizer(
    raw_dataset["train"][:2]['text'],
    truncation=True,
    max_length=Config.context_length,
    return_overflowing_tokens=True,
    return_length=True

)

print(f"Input IDs length: {len(outputs['input_ids'])}")
print(f"Input chunk lengths: {outputs['length']}")
print(f"Chunk mapping: {outputs['overflow_to_sample_mapping']}")

Input IDs length: 3
Input chunk lengths: [512, 390, 496]
Chunk mapping: [0, 0, 1]


In [41]:
raw_dataset['train'][:2]['text']

['ஃபாயெட்டெ (1825) ஆகியவற்றால் சாமுவேல் மோர்ஸுக்கு ஜனநாயக தேசிய உணர்வின் மீது குழப்பம் ஏற்பட்டது. செயல் முறையில் அமெரிக்க மக்களாட்சியைக் காண்பிப்பதற்காக அமெரிக்க பிரதிநிதிகள் அவையை அவர் ஓவியமாக வரையத் தேர்ந்தெடுத்தார். அவர் புதிய அறைகளின் கட்டமைப்பை வரைவதற்கு வாஷிங்டன் D.C.க்கு பயணித்தார். அந்த ஓவியத்தில் கவனமாக எண்பது பேரை வரைந்திருந்தார். மேலும் அதில் இடம் பெற்றிருந்த இரவுக்காட்சி பொருத்தமானதாக இருப்பதாக நம்பினார். அவர் வட்டமான மண்டபத்தின் விளக்குகளின் வெளிச்சத்தின் குவிதிறனுக்கு ஏற்ப அதில் இடம் பெற்றிருந்தோரை வெற்றிகரமாக சீராக வரைந்திருந்தார். இணைந்திருக்கும் மக்கள், தனியாக நின்றிருப்போர், தங்கள் மேஜைகளின் மீது சாய்ந்திருக்கும் தனியாளர் போன்றவை சாதாரணமாக வரையப்பட்டிருந்தது. ஆனால் சிறப்பியல்புடன் கூடிய முகங்களாக இருந்தது. சாமுவெல் மோர்ஸ் மக்களாட்சி கொள்கைகளின் மீது காங்கிரஸ்சின் அர்ப்பணிப்பு காலத்தின் அனுபவ வரம்பைத் தாண்டியதை வெளிப்படுத்துவதற்கு இரவு நேரத்தைத் தேர்ந்தெடுத்தார். எனினும் காங்கிரஸ் நியூயார்க் நகரத்தில் மக்கள் கூட்டத்தைப் பெறத் தவறியது. 1820 ஆம் ஆண்டில் ஜான் ட்ரம்புலின் 

In [44]:
def tokenize(element):
    outputs = tokenizer(
        element["text"],
        truncation=True,
        max_length=Config.context_length,
        return_overflowing_tokens=True,
        return_length=True
    )

    input_batch = []
    for length, input_ids in zip(
        outputs['length'], outputs['input_ids']
    ):
        if length == Config.context_length:
            input_batch.append(input_ids)
    return {"input_ids": input_batch}

tokenized_dataset = raw_dataset.map(
    tokenize,
    batched=True,
    remove_columns=["text"],
)

Map:   0%|          | 0/1184417 [00:00<?, ? examples/s]

Map:   0%|          | 0/345494 [00:00<?, ? examples/s]

In [45]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids'],
        num_rows: 1528958
    })
    test: Dataset({
        features: ['input_ids'],
        num_rows: 445316
    })
})

In [47]:
from transformers import AutoTokenizer, GPT2LMHeadModel, AutoConfig

config = AutoConfig.from_pretrained(
    "gpt2",
    vocab_size=len(tokenizer),
    n_ctx=Config.context_length,
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id,
)

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [48]:
model = GPT2LMHeadModel(config)
model_size = sum(t.numel() for t in model.parameters())
print(f"GPT-2 size: {model_size/1000**2:.1f}M parameters")

GPT-2 size: 108.9M parameters


In [ ]:
# 1. First, add the pad token more explicitly
if tokenizer.pad_token is None:
    # Method 1: Set pad_token to eos_token
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.eos_token_id
    
    # Method 2: If Method 1 doesn't work, try adding a special token
    # This is more reliable as it modifies the tokenizer's vocabulary
    special_tokens_dict = {'pad_token': '[PAD]'}
    num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)
    print(f"Added {num_added_toks} special tokens: {special_tokens_dict}")
    
    # If working with a model, resize embeddings to match new vocabulary size
    # model.resize_token_embeddings(len(tokenizer))

# 2. Verify that pad token is set
print(f"Pad token: '{tokenizer.pad_token}', ID: {tokenizer.pad_token_id}")

Added 1 special tokens: {'pad_token': '[PAD]'}
Pad token: '[PAD]', ID: 30000


In [64]:
# 2. Create data collator with explicit padding
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False, 
)

# 3. Try processing your batch
out = data_collator([tokenized_dataset["train"][i] for i in range(5)])
for key in out:
    print(f"{key} shape: {out[key].shape}")

input_ids shape: torch.Size([5, 512])
attention_mask shape: torch.Size([5, 512])
labels shape: torch.Size([5, 512])


In [65]:
from transformers import Trainer, TrainingArguments

args = TrainingArguments(
    output_dir="artifacts",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    evaluation_strategy="steps",
    eval_steps=5_000,
    logging_steps=5_000,
    gradient_accumulation_steps=8,
    num_train_epochs=1,
    weight_decay=0.1,
    warmup_steps=1_000,
    lr_scheduler_type="cosine",
    learning_rate=5e-4,
    save_steps=5_000,
    fp16=True,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=args,
    data_collator=data_collator,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
)

/home/tensorthiru/llm_pretrain/venv/lib/python3.10/site-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/tmp/ipykernel_20927/3361315117.py:21: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

OutOfMemoryError: CUDA out of memory. Tried to allocate 192.00 MiB. GPU 0 has a total capacity of 7.60 GiB of which 139.06 MiB is free. Process 2120 has 102.55 MiB memory in use. Including non-PyTorch memory, this process has 7.10 GiB memory in use. Of the allocated memory 6.92 GiB is allocated by PyTorch, and 55.95 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

: 